# sspa tutorial: Single sample pathway analysis in Python

### Installing and importing required packages
Please run all the cells in this section to ensure the following cells run smoothly

Install sspa (this is the testing version until we are happy with updates, then sspa v0.1.4 will be released on pip). If using the test version - don't need to run next cell. 

Install sspa

In [ ]:
!pip install sspa

Import requried python packages

In [ ]:
import sspa
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
pyo.init_notebook_mode()

## Load and process metabolomics data

Pre-processing example COVID mass spectrometry dataset

In [ ]:
covid_data = sspa.load_example_data(omicstype="metabolomics", processed=False)

In [ ]:
# Keep only metabolites (exclude metadata columns)
covid_values = covid_data.iloc[:, :-2]
# Remove features with too many na values
data_filt = covid_values.loc[:, covid_values.isin([' ', np.nan, 0]).mean() < 0.5]

# PQN normalisation
met_median = data_filt.median(axis=0, skipna=True)  # median value for each metabolite
scale_mat = data_filt.divide(met_median, axis=1)  # scale the matrix by the metabolite median
samp_median = scale_mat.median(axis=1, skipna=True)  # median value for each sample
norm_mat = data_filt.divide(samp_median, axis=0)  # scale by sample median

# Impute using the median
imputed_mat = norm_mat.fillna(norm_mat.median())
# Log transform the data
log2_mat = np.log2(imputed_mat)
# Standardise the data
processed_data = pd.DataFrame(StandardScaler().fit_transform(log2_mat), columns=imputed_mat.columns, index=imputed_mat.index)

In [ ]:
processed_data.head()

In [ ]:
covid_data['Group'].value_counts()

OPTIONAL: Save the processed data to .csv format

In [ ]:
processed_data.to_csv("example_covid_data_processed.csv")

Alternatively, to load the pre-processed dataset, set the argument "processed" to True

In [ ]:
covid_data_processed = sspa.load_example_data(omicstype="metabolomics", processed=True)

### Loading your own dataset

The easiest way to load your own metabolomics dataset to use with sspa is to use the pandas.read_csv() function to read in the CSV file. You can also read in sample metadata as part of the same file, or as a separate file, as long as all metadata is in the form of pandas Series/DataFrame columns. 

In [ ]:
# my_metabolomics_data = pd.read_csv("path/to/inputdata.csv")

## Identifier harmonisation (optional)

To map compounds in the metabolomics data to those in pathways, we must ensure the metabolite IDs in the data match those in the pathway database used. 

The MetaboAnalyst (Pang et al., 2021) ID conversion tool can convert to and from many input formats including compound names, HMDB ID, KEGG ID, PubChem, ChEBI, and METLIN. In this protocol we will be using the sspa.identifier_conversion() functions to retrieve the compound mapping from the MetaboAnalyst API, but this can also be done using the GUI (https://www.metaboanalyst.ca/MetaboAnalyst/upload/ConvertView.xhtml). 

This step can take several minutes depending on how many identifiers you wish to convert. 


The output ID can be any of 'HMDB', 'PubChem', 'ChEBI', 'KEGG', 'METLIN', or 'SMILES'. 

First we will create a compound ID conversion table using the sspa.identifier_conversion(input_id_type, identifier_list) function. This function takes two arguments, the first is the input identifier type (see Table 4 of the protocol for possible inputs), and a list of compound names/identifiers. The example COVID metabolomics data has compound names as identifiers, so we will specify the ‘input_type’ argument as ‘name’. 

**CRITICAL**: It is highly recommended to check that the input identifiers (‘identifier_list’) are clean and do not contain anomalies such as semicolons ‘;’ which may result in spurious matches or errors in the identifier conversion module. 


Use the identifier_conversion function to get identifier mappings. Change the input_type argument to the type of compound identifier you are converting from. This step may take a few minutes to complete.

In [ ]:
compound_names = processed_data.columns.tolist()
conversion_table = sspa.identifier_conversion(input_type="name", compound_list=compound_names)

In [ ]:
conversion_table

Count how many identifiers have matches

In [ ]:
conversion_table["Comment"].value_counts()

OPTIONAL: export conversion table as a .csv file, manually update missing identifiers and re-upload the conversion table to proceed with the ID mapping step

**Only run the following two cells if you wish to manually edit the ID matching results**

In [ ]:
conversion_table.to_csv("conversion_table.csv")

In [ ]:
conversion_table = pd.read_csv("conversion_table.csv", index_col=0)

We can then use the sspa.map_identifiers(conversion_table, output_id_type, data_matrix) function to convert the identifiers in our dataset. We must specify three arguments to the function: the conversion table generated in the previous step, the output ID type, and the matrix containing the metabolomics data. This function will return the metabolomics data matrix with mapped identifiers.

In [ ]:
processed_data_mapped = sspa.map_identifiers(conversion_table, output_id_type="ChEBI", matrix=processed_data)

In [ ]:
processed_data_mapped

## Importing pathways

Sspa comes with pre-loaded versions of Reactome (all supported organisms) release 78 (2022) and KEGG (human) release 98 (2022). It also allows users to easily download the latest version of KEGG and Reactome (all supported organisms). Users can also provide their own pathway files, as long as they are in GMT format. 

The sspa package has several functions for processing data for different databases. These are:
-	Process_reactome(organism, infile=None, download_latest=False, filepath=None)
-	Process_kegg(organism, infile=None, download_latest=False, filepath=None)
-	Process_gmt(infile) - load files in GMT format, ending in .gmt or .csv extension

For KEGG and Reactome, you will need to specify the organism code/name in the function call. KEGG 3-letter organism codes e.g. “hsa”, “mmu”, can be found at http://rest.kegg.jp/list/organism. Reactome organism names e.g. “Homo sapiens”, “Mus musculus”, can be found at https://reactome.org/content/schema/objects/Species. 


### Using pre-loaded pathway databases

Import the pre-loaded Reactome metabolic pathways (Release 78)

In [ ]:
# We will import the metabolite pathways from the Reactome database
# We must specify one of the Reactome organism names
# This returns a GMT format pandas DataFrame containing the pathway information
reactome_pathways  = sspa.process_reactome(organism="Homo sapiens")

In [ ]:
kegg_human_pathways  = sspa.process_kegg(organism="hsa")

In [ ]:
reactome_pathways.head()

In [ ]:
kegg_human_pathways.head()

### Specifying your own pathways (in GMT format)

In [ ]:
# download example GMT filt - Wikipathways Homo sapiens
import urllib.request

urllib.request.urlretrieve("https://wikipathways-data.wmcloud.org/current/gmt/wikipathways-20220310-gmt-Homo_sapiens.gmt",
                  "wikipathways-20220310-gmt-Homo_sapiens.gmt")

# load the GMT file using sspa
custom_pathways = sspa.process_gmt("wikipathways-20220310-gmt-Homo_sapiens.gmt")

In [ ]:
custom_pathways

## Downloading latest versions of KEGG and Reactome databases and MetExplore pathway networks

### Download latest KEGG pathways

In [ ]:
kegg_mouse_latest = sspa.process_kegg("mmu", download_latest=True, filepath=".")

### Download latest Reactome pathways

In [ ]:
# download Reactome latest
reactome_mouse_latest = sspa.process_reactome("Mus musculus", download_latest=True, filepath=".")

### Download latest metabolic network pathways from MetExplore

The possible identifier types are:


*   "chebi"
*   "kegg"



In [ ]:
ihuman = sspa.MetExplorePaths(model='6422', id_type="chebi", filepath=None)

In [ ]:
print(ihuman.nMetab, "total metabolites in the metabolic network")
print(ihuman.nMappedID, "metabolites with ChEBI identifiers")
coverage = int(ihuman.nMappedID)/int(ihuman.nMetab)*100
print("the coverage of metabolites in term of ChEBI identifiers for the network is", round(coverage,1),"%")

In [ ]:
ihuman.pathways.head()

Read in a .gmt file (here we use the latest Reactome pathways downloaded in the cell above)

In [ ]:
reactome_mouse_latest_read = sspa.process_gmt("./Reactome_Mus_musculus_pathways_compounds_R79.gmt")

In [ ]:
reactome_mouse_latest_read

## Checking ID mappings
Check how many of the mapped compound identifiers are in the pathway database (i.e. annotated to a pathway):

In [ ]:
## if using Reactome database as the pathway database
# count all compounds in the dataset
print(len(compound_names), "compounds in the dataset")

# find how many input compound names in the dataset had a matching ChEBI ID
chebi_matches = conversion_table[(conversion_table["Comment"] == "1") & (conversion_table["ChEBI"].isnull()==False)]["ChEBI"]
print(len(chebi_matches), "compounds from the dataset that have ChEBI IDs")

# count all unique compounds in the Reactome database
all_reactome_cpds = set(sum(sspa.utils.pathwaydf_to_dict(reactome_pathways).values(), []))
print(len(all_reactome_cpds), "total unique compounds in Reactome")

# find the intesect between all reactome compounds and all ChEBI IDs annotated to the dataset
mapped_annotated_cpds = set(processed_data_mapped.columns) & all_reactome_cpds
print(len(mapped_annotated_cpds), "compounds present in both the dataset and Reactome pathways")

In [ ]:
## if using the metabolite network ihuman1.10 as the pathway database
# count all compounds in the dataset
print(len(compound_names), "compounds in the dataset")

# find how many input compound names in the dataset had a matching ChEBI ID
chebi_matches = conversion_table[(conversion_table["Comment"] == "1") & (conversion_table["ChEBI"].isnull()==False)]["ChEBI"]
print(len(chebi_matches), "compounds from the dataset that have ChEBI IDs")

# get the total number of metabolites in the network
print(ihuman.nMetab, "total compounds in the metabolic network")

# get the total number of metabolites with ChEBI in the network
print(ihuman.nChEBI, "total compounds with ChEBI identifiers in the metabolic network")

# count all unique ChEBI in the metabolic network
all_ihuman_ChEBI = set(sum(sspa.utils.pathwaydf_to_dict(ihuman.pathways).values(), []))
print(len(all_ihuman_ChEBI), "total unique ChEBI identifiers in metabolic network")

# find the intesect between all network compounds and all ChEBI IDs annotated to the dataset
mapped_annotated_cpds = set(processed_data_mapped.columns) & all_ihuman_ChEBI
print(len(mapped_annotated_cpds), "compounds present in both the dataset and network pathways")

Visualise this in barchart form:

In [ ]:
sns.set_context('notebook')
sns.set_style('ticks')
sns.barplot(y=[len(compound_names), len(chebi_matches), len(mapped_annotated_cpds)], x=['Original', 'Mapping to CHEBI', 'Annotated to Reactome'])
plt.tight_layout()
plt.show()

Or funnel plot...

In [ ]:
data = dict(count=[len(compound_names), len(chebi_matches), len(mapped_annotated_cpds)],
            label=['Original', 'Mapping to CHEBI', 'Annotated to Reactome'])

fig = px.funnel(data, x='count', y='label')
fig.show()

Here we plot an interactive heatmap that shows which compounds have mapped to pathway database IDs, and those that are present in pathways. Hover over the plot or **zoom in** to see the compound names.

In [ ]:
import plotly.io as pio
pio.renderers.default = "notebook"

df = pd.DataFrame(compound_names, columns=['Original_ID'])
df["Matched_ID"] = df['Original_ID'].map(dict(zip(conversion_table["Query"], conversion_table["ChEBI"])))
df["In_pathway"] = [i if i in mapped_annotated_cpds else "NA" for i in df["Matched_ID"] ]
df = df.replace({"NA":0})
df[df != 0] = 1
df = df.astype("float")
df.index = compound_names

fig = px.bar(df)
fig.show()

How many pathways contain at least two mapped compounds?

In [ ]:
# convert the pathway dataframe to dictionary - for faster calculations
# replace reactome_pathways with the variable containing the pathway dataframe being used
pathways_dict = sspa.utils.pathwaydf_to_dict(reactome_pathways)

# How many pathways contain at least two mapped compounds?
pathways_present = {k: v for k, v in pathways_dict.items() if len([i for i in processed_data_mapped.columns if i in v]) > 1}
print(len(pathways_present))

## Conventional pathway analysis methods

### ORA

1. Identify the background set to be used. By default, sspa uses all the compounds annotated in the input metabolomics data (i.e. all the column identifiers) as the background set. Alternatively, the sspa_ora() class allows the user to specify a custom background set of compounds to be used. This must be passed to the class using the argument ‘custom_background’ (see step 2) in the form of a python list, containing compound identifiers specific to the pathway database used. 

2. Initiate an object of the sspa_ora class with the required parameters. The class parameters are the processed metabolomics DataFrame with identifiers corresponding to those in the pathway database to be used (processed_data_mapped), a pandas Series containing the sample metadata (covid_data["Group"]), pathway database, the cutoff alpha threshold for differential abundance analysis, and an optional custom background set. Here we are using $a$ = 0.05 to select differentially abundant metabolites, and all the metabolites in the processed abundance matrix as the background set (default).  This step will also perform differential abundance analysis using t-tests. 


In [ ]:
# initiate an ORA object 
ora = sspa.sspa_ora(processed_data_mapped, covid_data["Group"], reactome_pathways, 0.05, custom_background=None)

# perform ORA 
ora_res = ora.over_representation_analysis()

In [ ]:
ora.DA_molecules

In [ ]:
ora.ttest_res.sort_values(by="P-value")

Now we can examine the table containing the ORA results:

In [ ]:
ora_res

In [ ]:
ora_res.sort_values(by="P-value")

In [ ]:
top_20_pathways = ora_res.sort_values(by="P-value").iloc[0:20, :]
plt.figure(figsize=(8, 6))
sns.barplot(data=top_20_pathways, y="Pathway_name", x="P-value", orient="h", palette="magma")
plt.axvline(0.05, c="black")
# plt.savefig("ORA_top_20.png", dpi=300, bbox_inches="tight")
plt.show()

### GSEA
GSEA is a popular method for pathway analysis in the transcriptomics community (Subramanian et al., 2005). The sspa package has a wrapper function to allow users to use the R-based fast implementation of GSEA, fGSEA (Korotkevich, Sukhov and Sergushichev, 2019). The sspa function sspa_fgsea uses the signal-to-noise ratio as the metabolite ranking metric for GSEA. We encourage users to read about various ranking metrics available (Zyla et al., 2017) and experiment with the fGSEA package if they wish to use other ranking metrics. 

Run gsea using the sspa_fgsea() function. This function requires the processed metabolomics data, a pandas series containing the sample metadata, and the pathways DataFrame as input. The ranking metric used is the signal-to-noise ratio. 


In [ ]:
gsea_res = sspa.sspa_fgsea(processed_data_mapped, covid_data['Group'], reactome_pathways)

In [ ]:
gsea_res.sort_values(by="P-value")

Plot the GSEA results - GSEA provides us with an enrichment score which can be positive or negative, this is highlighted using the bar colour. 


In [ ]:
from matplotlib.lines import Line2D

top_20_pathways_gsea = gsea_res.sort_values(by="P-value").iloc[0:20, :]
plt.figure(figsize=(8, 6))

# set bar colour based on normalised enrichment score sign
bar_color = ['tab:red' if float(i) < 0 else 'tab:blue' for i in top_20_pathways_gsea['NES']]
sns.barplot(data=top_20_pathways_gsea, y="Pathway_name", x="P-value", orient="h", palette=bar_color)
plt.axvline(0.05, c="black")

# add legend
custom_lines = [Line2D([0], [0], color='tab:red', lw=4),
                Line2D([0], [0], color='tab:blue', lw=4)]
plt.legend(handles=custom_lines, labels=['Positive enrichment score', 'Negative enrichment score'])
plt.show()

# plt.savefig("GSEA_top_20.png", dpi=300, bbox_inches="tight")

## Single-sample pathway analysis methods

Using kPCA method

In [ ]:
kpca_scores = sspa.sspa_kpca(processed_data_mapped, reactome_pathways)

In [ ]:
# Inspect the pathway score matrix
kpca_scores

### Visualise single-sample pathway analysis results

Pathway-based PCA plot

In [ ]:
# Pathway-based PCA plot

# Normalise kPCA scores
kpca_scores_norm = pd.DataFrame(StandardScaler().fit_transform(kpca_scores))

# Perform two component PCA using sklearn
pca = PCA(n_components=2)
pca_res = pca.fit_transform(kpca_scores_norm)

# determine the variance explained by the first 2 components
pca.explained_variance_ratio_

# Plot the first two components as a scatterplot
plt.style.use("default")
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)
sns.scatterplot(x=pca_res[:, 0 ], y=pca_res[:, 1], hue=covid_data["Group"], ax=ax1, s=50, alpha=0.5)
sns.scatterplot(x=pca_res[:, 0 ], y=pca_res[:, 1], hue=covid_data["WHO_status"], ax=ax2, s=50, alpha=0.5)

# Set axis labels 
ax1.set_xlabel('PC1 (' + str(round(pca.explained_variance_ratio_[0]*100,2)) + '%)')
ax1.set_ylabel('PC2 (' + str(round(pca.explained_variance_ratio_[1]*100,2)) + '%)')
ax2.set_xlabel('PC1 (' + str(round(pca.explained_variance_ratio_[0]*100,2)) + '%)')
ax2.set_ylabel('PC2 (' + str(round(pca.explained_variance_ratio_[1]*100,2)) + '%)')

plt.tight_layout()
# plt.savefig(".kpca_pca_plots.png", dpi=350, bbox_inches="tight")

plt.show()

PCA loadings based on pathways - hover over datapoints to see the pathway name associated with the loading

In [ ]:
# get the loadings of the pathway-based PCA
loadings = pd.DataFrame(pca.components_.T* np.sqrt(pca.explained_variance_)*10,columns=['PC1','PC2'], index=kpca_scores.columns)

# add pathway names to the loadings dataframe
loadings['Pathway'] = loadings.index.map(dict(zip(reactome_pathways.index, reactome_pathways['Pathway_name'])))

# subset top 10 loadings for visual clarity
loadings_top_10 = loadings.sort_values(by='PC1').iloc[0:10, :]

# Plot the first two components as a scatterplot
fig = px.scatter(x=pca_res[:, 0 ], y=pca_res[:, 1], color=covid_data["Group"], labels={'x':'PC1', 'y':'PC2'})

# Plot lines to origin representing the loadings
for i in range(0, loadings_top_10.shape[0]):
  fig.add_trace(go.Scatter(x=[0, loadings_top_10.iloc[i, :]['PC1']], y=[0, loadings_top_10.iloc[i, :]['PC2']],
                           line_color='black', marker_size=0, text=loadings_top_10.iloc[i, :]['Pathway']))

fig.update_layout(width=600, height=600, yaxis_range=[-10, 10], xaxis_range=[-15, 15], showlegend=False)

fig.show()


Use pathway scores to plot a hierarchical clustering heatmap

In [ ]:
# Plot a heatmap using the pathway scores
g = sns.clustermap(kpca_scores_norm.T,
               cmap="RdBu_r",
               z_score=1,
              col_colors = ["tab:red" if i == "COVID19 " else "tab:green" for i in covid_data["Group"]],
              xticklabels=False,
              yticklabels=False)
g.ax_heatmap.set_xlabel("Samples")
g.ax_heatmap.set_ylabel("Pathways")

# plt.savefig("kpca_heatmap.png", dpi=350, bbox_inches="tight")
plt.show()

In [ ]:
# Plot pathway scores from two pathways against each other
P1 = "R-HSA-392499"
P2 = "R-HSA-425397"
plt.style.use("default")
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)
sns.scatterplot(x=kpca_scores.loc[:, P1], y=kpca_scores.loc[:, P2], hue=covid_data["Group"], ax=ax1, s=100, alpha=0.5)
sns.scatterplot(x=kpca_scores.loc[:, P1], y=kpca_scores.loc[:, P2], hue=covid_data["WHO_status"], ax=ax2, s=100, alpha=0.5)
# Set axis labels 
ax1.set_xlabel(P1 + " - " + reactome_pathways.loc[P1]["Pathway_name"])
ax1.set_ylabel(P2 + " - " + reactome_pathways.loc[P2]["Pathway_name"])
ax2.set_xlabel(P1 + " - " + reactome_pathways.loc[P1]["Pathway_name"])
ax2.set_ylabel(P2 + " - " + reactome_pathways.loc[P2]["Pathway_name"])

### Additional sspa methods

Using ssClustPA method

In [ ]:
ssclustpa_res = sspa.sspa_cluster(processed_data_mapped, reactome_pathways)

In [ ]:
ssclustpa_res.head()

Using z-score method

In [ ]:
zscore_res = sspa.sspa_zscore(processed_data_mapped, reactome_pathways)

In [ ]:
zscore_res.head()

Using SVD (PLAGE) method

In [ ]:
svd_res = sspa.sspa_svd(processed_data_mapped, reactome_pathways)
svd_res.head()

Using GSVA (Hanzelmann et al)

In [ ]:
gsva_res = sspa.sspa_gsva(processed_data_mapped, reactome_pathways)
gsva_res.head()

Using ssGSEA (Barbie et al)

In [ ]:
ssgsea_res = sspa.sspa_ssGSEA(processed_data_mapped, reactome_pathways)
ssgsea_res.head()